In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
from langchain.tools import tool
from typing import Dict, Any
from tavily import TavilyClient

tavily_client = TavilyClient()

@tool
def web_search(query: str) -> Dict[str, Any]:
    """
    Search the web for information
    """

    return tavily_client.search(query)

In [3]:
system_prompt = """
    You are a personal chef. The user will give you a list of ingredients they have left over in their house.

    Using the web search tool, search the web for recipes that can be made with the ingredients they have.

    Return recipe suggestions and eventually the recipe instructions to the user, if requested.
"""

In [4]:
from langchain.agents import create_agent
from langgraph.checkpoint.memory import InMemorySaver

agent = create_agent(
    model="gpt-5-nano",
    tools=[web_search],
    system_prompt=system_prompt,
    checkpointer=InMemorySaver()
)

In [5]:
from langchain.messages import HumanMessage

config = {"configurable": {"thread_id": "1"}}

response = agent.invoke(
    {"messages": [HumanMessage(content="I have some leftover chicken and rice. What can I make?")]},
    config
)

print(response["messages"][-1].content)

Nice—leftover chicken and rice are perfect for quick, comforting meals. Here are several solid options you can choose from (all fairly common to make and great with basic pantry staples):

- One-Pot Creamy Chicken and Rice (stovetop)
  - Why it’s great: Cozy, creamy texture with minimal cleanup; everything comes together in one pot.
  - Approx time: 30–40 minutes.
  - Source idea: Budget Bytes / The Dinner Bite style recipes.

- Chicken Fried Rice (skillet)
  - Why it’s great: Uses leftover rice crisps up nicely and you can toss in any veggies you’ve got. Very quick.
  - Approx time: 15–20 minutes.
  - Source idea: Classic fried rice recipes (RecipeTin Eats, Averie Cooks, etc.).

- Chicken and Rice Casserole (baked)
  - Why it’s great: Hearty, comforting, and easy to bake in one dish; you can add peas or broccoli for color and nutrition.
  - Approx time: 45–60 minutes (plus bake time).
  - Source idea: Various “one-pan” or “casserole” chicken and rice recipes.

- Chicken and Rice Soup 

In [6]:
from pprint import pprint

pprint(response)

{'messages': [HumanMessage(content='I have some leftover chicken and rice. What can I make?', additional_kwargs={}, response_metadata={}, id='7feb186f-b92b-4b69-a1a8-87466b803089'),
              AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 768, 'prompt_tokens': 202, 'total_tokens': 970, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 640, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-5-nano-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-CsZtSPTtc30s8E3bWHP3siptAmU87', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019b70c8-1f9b-7960-8bba-1d0f0c02bf10-0', tool_calls=[{'name': 'web_search', 'args': {'query': 'leftover chicken and rice recipe'}, 'id': 'call_6bQ43MsVkalBUE8vgsFCUCnu', 'type': 'tool_call'}, {'name': 

https://smith.langchain.com/public/6a492840-4258-4013-a079-8969b03b5ac1/r

## Follow up question using an image

In [7]:
from ipywidgets import FileUpload
from IPython.display import display

uploader = FileUpload(accept=".png", multiple=False) # Use /files/refrigerator.png
display(uploader)

FileUpload(value=(), accept='.png', description='Upload')

In [8]:
print(uploader.value)

({'name': 'refrigerator.png', 'type': 'image/png', 'size': 38439, 'content': <memory at 0x131f02680>, 'last_modified': datetime.datetime(2025, 12, 30, 19, 41, 4, 934000, tzinfo=datetime.timezone.utc)},)


In [9]:
import base64

# Get the first (and only) uploaded file dict
uploaded_file = uploader.value[0]

# This is memoryview
content_mv = uploaded_file["content"]

# Convert memoryview -> bytes
img_bytes = bytes(content_mv) # or content_mv.tobytes()

# Now base64 encode
img_b64 = base64.b64encode(img_bytes).decode("utf-8") # Convert raw bytes into base64 text string.

In [10]:
multimodal_question = HumanMessage(
    content=[
        {"type": "text", "text": 
            """
                Tell me what you notice about my refrigerator here?

                Also, from the recipes that you provided, which one is the easiest
                for me to do if I need to store them in my refrigerator?
            """},
        {"type": "image", "base64": img_b64, "mime_type": "image/png"}
    ]
)

response = agent.invoke(
    {"messages": [multimodal_question]},
    config
)

print(response["messages"][-1].content)

Nice, I’ll tackle both parts.

What I notice about your refrigerator (from the image)
- It looks like a compact/apartment-size fridge with a standard interior layout (shelves, a crisper drawer at the bottom, and door shelves).
- The unit appears clean and mostly empty, which is great for planning leftovers.
- The door swings open to the left in the photo, with multiple door trays for bottles or small jars.
- There isn’t visible produce or dairy inside, so you’ve got space to add a few containers of leftovers or prepared meals.
- Overall, it’s a basic, no-frills fridge that’s easy to organize if you portion food into airtight containers.

Which of the recipes is the easiest to store in the fridge
- My pick for easiest storage: Chicken Rice Casserole (the “one-pan baked” option).
  - Why: It’s a single dish that you can portion out into airtight containers or keep in the baking dish with a lid. It stores well in the fridge for 3–4 days and reheats nicely in the microwave or oven without 

In [11]:
pprint(response)

{'messages': [HumanMessage(content='I have some leftover chicken and rice. What can I make?', additional_kwargs={}, response_metadata={}, id='7feb186f-b92b-4b69-a1a8-87466b803089'),
              AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 768, 'prompt_tokens': 202, 'total_tokens': 970, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 640, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-5-nano-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-CsZtSPTtc30s8E3bWHP3siptAmU87', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019b70c8-1f9b-7960-8bba-1d0f0c02bf10-0', tool_calls=[{'name': 'web_search', 'args': {'query': 'leftover chicken and rice recipe'}, 'id': 'call_6bQ43MsVkalBUE8vgsFCUCnu', 'type': 'tool_call'}, {'name': 

https://smith.langchain.com/public/bf9cb10c-2085-4dc3-887d-4dd771182643/r